In [88]:
# Reading and data and display sample

import pandas as pd
import os
import torch
from transformers import BertTokenizer
from datasets import load_dataset, load_metric
import time
import json
from pathlib import PurePath
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from sklearn.model_selection import train_test_split


device = "cuda" if torch.cuda.is_available() else "cpu"

print(f'device is {device}')


device is cpu


In [89]:
pl.seed_everything(42)

Global seed set to 42


42

In [43]:
file_path = 'data/nikhil_onlytextpytorch.json'

path = os.path.join(os.getcwd(),file_path)

train = pd.read_json(path, orient='index').reset_index()
display(train)

,index,answer,question,context,source
0,"What values are specified to replaceNaN, posit...","bynan,posinf, andneginf","What values are specified to replaceNaN, posit...","ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
1,"By default,NaN is replaced with what value?",zero,"By default,NaN is replaced with what value?","ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
2,What values are used to replace negative infin...,"bynan,posinf, andneginf",What values are used to replace negative infin...,"ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
3,What is the default value for negative infinity?,the least finite value,What is the default value for negative infinity?,"ReplacesNaN, positive infinity, and negative i...",https://pytorch.org/docs/stable/generated/torc...
4,What is the value to replace positive infinity...,posinf,What is the value to replace positive infinity...,"nan(Number,optional) – the value to replaceNaN...",https://pytorch.org/docs/stable/generated/torc...
...,...,...,...,...,...
998,What is the tensor of shape where*is zero or m...,A(Tensor),What is the tensor of shape where*is zero or m...,torch.linalg.inv()computes the inverse of a sq...,https://pytorch.org/docs/stable/generated/torc...
999,What is the tolerance value to determine when ...,rcond,What is the tolerance value to determine when ...,torch.linalg.lstsq()computesA.pinv() @Bwith a\...,https://pytorch.org/docs/stable/generated/torc...
1000,What is the default value of atorch.Tensor?,1e-15,What is the default value of atorch.Tensor?,See also torch.linalg.inv()computes the invers...,https://pytorch.org/docs/stable/generated/torc...
1001,What computes the inverse of a square matrix?,torch.linalg.inv(),What computes the inverse of a square matrix?,See also torch.linalg.inv()computes the invers...,https://pytorch.org/docs/stable/generated/torc...


In [45]:
train = (train[['question', 'answer']])


In [46]:
display(train)

,question,answer
0,"What values are specified to replaceNaN, posit...","bynan,posinf, andneginf"
1,"By default,NaN is replaced with what value?",zero
2,What values are used to replace negative infin...,"bynan,posinf, andneginf"
3,What is the default value for negative infinity?,the least finite value
4,What is the value to replace positive infinity...,posinf
...,...,...
998,What is the tensor of shape where*is zero or m...,A(Tensor)
999,What is the tolerance value to determine when ...,rcond
1000,What is the default value of atorch.Tensor?,1e-15
1001,What computes the inverse of a square matrix?,torch.linalg.inv()


In [47]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

questions = tokenizer(list(train['question']), truncation=True, padding=True)
answers = tokenizer(list(train['answer']), truncation=True, padding=True)

train_labels = []

In [83]:
# Lets decode and have a look at the sample dataset

for x in zip(questions["input_ids"][:2], answers["input_ids"][:2]):
    print(tokenizer.decode(x[0]), tokenizer.decode(x[1]))

[CLS] what values are specified to replacenan, positive infinity, and negative infinity values in input? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [CLS] bynan, posinf, andneginf [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] by default, nan is replaced with what value? [SEP] [PAD] [PAD] [PAD] [PA

In [55]:
question_encoding = tokenizer(
    train['question'][0],
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt")

In [56]:
answer_encoding = tokenizer(
    train['answer'][0],
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt")

In [57]:
question_encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [58]:
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [59]:
tokenizer.decode(question_encoding['input_ids'].squeeze())

'[CLS] what values are specified to replacenan, positive infinity, and negative infinity values in input? [SEP]'

In [60]:
tokenizer.decode(answer_encoding['input_ids'].squeeze())

'[CLS] bynan, posinf, andneginf [SEP]'

In [81]:
import torch

class text_tokenizer(torch.utils.data.Dataset):
    def __init__(
        self,
        data:pd.DataFrame, 
        tokenizer:BertTokenizer,
        column_name:str,
        source_max_length: int=30,
        ):

        self.tokenizer = tokenizer
        self.data = data
        self.source_max_length = source_max_length
        self.column_name = column_name


    def __len__(self):
        return len(self.data)


    def __getitem__(self, index:int):
        data_row = self.data.iloc[index]
        encoding = tokenizer(
            data_row[self.column_name],
            max_length=self.source_max_length,
            padding='max_length',
            return_attention_mask=True,
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")


        return dict(
            text=data_row[self.column_name],
            input_ids=encoding['input_ids'].flatten(),
            attention_mask=encoding['attention_mask'].flatten()
        )


In [82]:
token_text = text_tokenizer(train, tokenizer, column_name='question')

In [91]:
for data in token_text:
    print(data)
    break

{'text': 'What values are specified to replaceNaN, positive infinity, and negative infinity values in input?', 'input_ids': tensor([  101,  2054,  5300,  2024,  9675,  2000,  5672,  7229,  1010,  3893,
        15579,  1010,  1998,  4997, 15579,  5300,  1999,  7953,  1029,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])}


In [90]:
train_df, val_df = train_test_split(train, test_size=0.30)

In [95]:
class QnaLoaders(pl.LightningDataModule):

    def __init__(
        self,
        train_df:pd.DataFrame,
        val_df:pd.DataFrame,
        column_name:str,
        tokenizer=tokenizer,
        batch_size:int=8,
        max_token_length:int=256
        ):

        super().__init__()
        self.batch_size=batch_size
        self.train_df = train_df
        self.val_df = val_df,
        self.tokenizer = tokenizer
        self.max_token_length=max_token_length
        self.column_name = column_name
        
    def setup(self):
        self.train_dataset = text_tokenizer(
            self.train_df,
            self.tokenizer,
            self.column_name,
            self.max_token_length  
        )

        self.val_dataset = text_tokenizer(
            self.val_df,
            self.tokenizer,
            self.column_name,
            self.max_token_length  
        )
    
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4
        )
    

In [96]:
BATCH_SIZE = 32
N_EPOCHS = 10

data_module = QnaLoaders(train_df=train_df, val_df=val_df, tokenizer=tokenizer, column_name='question', batch_size=BATCH_SIZE, max_token_length=30)

In [97]:
data_module.setup()
